In [1]:
import time
import sys
import random
import qrcode
from projectq import MainEngine
from projectq.ops import H, Measure, All, CNOT, X
from projectq.backends import Simulator
from projectq.cengines import TagRemover
from pyzbar.pyzbar import decode
from PIL import Image

# Error rate simulation parameter
ERROR_RATE = 0.05  # 5% chance of error on each qubit

# Helper functions for QR Code handling
def generate_qr_code(data, title):
    qr = qrcode.make(data)
    qr.save(f"{title}.png")
    qr.show(title=title)
    return f"{title}.png"

def read_qr_code(filename):
    with Image.open(filename) as img:
        decoded_data = decode(img)[0].data.decode("utf-8")
    return decoded_data

# Quantum Fourier Transform
def apply_qft(eng, qubits):
    n = len(qubits)
    for i in range(n):
        H | qubits[i]
        for j in range(i + 1, n):
            CNOT | (qubits[j], qubits[i])

# Inverse Quantum Fourier Transform
def apply_inverse_qft(eng, qubits):
    n = len(qubits)
    for i in reversed(range(n)):
        for j in reversed(range(i + 1, n)):
            CNOT | (qubits[j], qubits[i])
        H | qubits[i]

# Introduce errors on qubits
def introduce_errors(qubits):
    errors = 0
    for qubit in qubits:
        if random.random() < ERROR_RATE:
            X | qubit  # Flip the qubit as an "error"
            errors += 1
    return errors

# Double AES with QFT for two 8-bit keys and QR code process
def Double_AES_QFT(eng, resource_check):
    # Allocate 8-bit registers
    x = eng.allocate_qureg(8)   # 8-bit plaintext
    k1 = eng.allocate_qureg(8)  # First 8-bit key
    k2 = eng.allocate_qureg(8)  # Second 8-bit key

    # Convert plaintext to QR code, display, and read back
    plaintext_data = "Sample plaintext message"
    plaintext_qr_filename = generate_qr_code(plaintext_data, "Plaintext QR Code")
    plaintext_from_qr = read_qr_code(plaintext_qr_filename)
    print("Text from Plaintext QR Code:", plaintext_from_qr)

    # Apply QFT to plaintext and keys
    print("Applying QFT to plaintext and keys...")
    apply_qft(eng, x)
    apply_qft(eng, k1)
    apply_qft(eng, k2)
    eng.flush()

    # First AES-8 Layer
    print("Running First AES Layer (AES-8)")
    AES_8(eng, resource_check, x, k1)

    # Convert intermediate ciphertext to QR code, display, and read back
    first_layer_ciphertext = "First layer ciphertext representation"  # Placeholder
    first_layer_ciphertext_qr_filename = generate_qr_code(first_layer_ciphertext, "First Layer Ciphertext QR Code")
    first_layer_ciphertext_from_qr = read_qr_code(first_layer_ciphertext_qr_filename)
    print("Ciphertext from First Layer QR Code:", first_layer_ciphertext_from_qr)

    # Introduce errors after the first AES layer
    print("Introducing errors after first AES layer...")
    errors1 = introduce_errors(x)

    # Apply inverse QFT
    print("Applying Inverse QFT to ciphertext and keys...")
    apply_inverse_qft(eng, x)
    apply_inverse_qft(eng, k1)
    apply_inverse_qft(eng, k2)
    eng.flush()

    # Second AES-8 Layer
    print("Running Second AES Layer (AES-8)")
    AES_8(eng, resource_check, x, k2)

    # Convert final ciphertext to QR code, display, and read back
    final_ciphertext = "Final ciphertext representation"  # Placeholder
    final_ciphertext_qr_filename = generate_qr_code(final_ciphertext, "Final Ciphertext QR Code")
    final_ciphertext_from_qr = read_qr_code(final_ciphertext_qr_filename)
    print("Ciphertext from Final QR Code:", final_ciphertext_from_qr)

    # Introduce errors after the second AES layer
    print("Introducing errors after second AES layer...")
    errors2 = introduce_errors(x)

    # Final Inverse QFT
    print("Applying final Inverse QFT to output...")
    apply_inverse_qft(eng, x)
    eng.flush()

    print("Final Ciphertext:")
    print_state(eng, x)

    return x, errors1 + errors2

# AES-8 function with an 8-bit key
def AES_8(eng, resource_check, x, k):
    Round_constant_XOR(eng, x, 0x12, 8)
    Round_constant_XOR(eng, k, 0x12, 8)

    for i in range(2):  # Two rounds for AES-8
        Keyshedule_8(eng, k, i, resource_check)
        SBox_bp12_all(eng, x, resource_check)
        if i != 1:
            x = Mini_mc(eng, x)
        AddRoundkey(eng, x, k)

    return x

# Helper functions
def Keyshedule_8(eng, k, round, resource_check):
    pass

def SBox_bp12_all(eng, x, resource_check):
    pass

def Mini_mc(eng, x):
    return x

def AddRoundkey(eng, x, k):
    CNOT8(eng, k, x)

def Round_constant_XOR(eng, x, constant, bits):
    for i in range(bits):
        if (constant >> i) & 1:
            X | x[i]

def CNOT8(eng, a, b):
    for i in range(8):
        CNOT | (a[i], b[i])

# Measure and print final qubit state
def print_state(eng, qubits):
    All(Measure) | qubits
    eng.flush()
    
    state = [int(qubit) for qubit in qubits]
    binary_str = ''.join(str(bit) for bit in state)
    hex_str = hex(int(binary_str, 2))[2:].zfill(len(qubits) // 4)
    
    print(f'Hex: {hex_str.upper()}')
    print(f'Binary: {binary_str}')
    sys.stdout.flush()

# Main function to run Double AES with QFT and QR code integration
def main():
    start_time = time.time()

    sim = Simulator()
    eng = MainEngine(backend=sim, engine_list=[TagRemover()])

    final_state, total_errors = Double_AES_QFT(eng, 0)
    
    # Calculate error rate
    num_qubits = 8
    error_rate = total_errors / (2 * num_qubits)
    print(f"\nTotal Errors Introduced: {total_errors}")
    print(f"Error Rate: {error_rate:.2%}")

    end_time = time.time()
    execution_time = end_time - start_time
    print(f'Execution Time: {execution_time:.4f} seconds')
    sys.stdout.flush()

if __name__ == '__main__':
    main()


(Note: This is the (slow) Python simulator.)
Text from Plaintext QR Code: Sample plaintext message
Applying QFT to plaintext and keys...
Running First AES Layer (AES-8)
Ciphertext from First Layer QR Code: First layer ciphertext representation
Introducing errors after first AES layer...
Applying Inverse QFT to ciphertext and keys...
Running Second AES Layer (AES-8)
Ciphertext from Final QR Code: Final ciphertext representation
Introducing errors after second AES layer...
Applying final Inverse QFT to output...
Final Ciphertext:
Hex: 3A
Binary: 00111010

Total Errors Introduced: 0
Error Rate: 0.00%
Execution Time: 1913.1563 seconds
